# Welcome to the Lab 🥼🧪

#### How to Download Data as a CSV?

In this notebook, you will learn how to use the Parcl Labs API to search for markets and download data as CSV files. This guide will help you:

1. **Search for Markets:** Understand how to use the search feature to find markets and retrieve their unique identifiers (`parcl_ids`).
2. **Retrieve Data:** Use these `parcl_ids` to pull various types of data about the markets, including housing metrics, investor metrics, portfolio metrics, and more.
3. **Download CSV Files:** Save the retrieved data as CSV files for further analysis and use in tools like Excel.

The Parcl Labs API includes over 70,000 markets, each identified by a unique `parcl_id`. The search feature allows you to retrieve these `parcl_ids`, which you can then use to pull detailed data about specific markets using other API methods.

**Reminders:**

- You can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

- To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this Google Colab [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.
- To run this notebook at scale and download data for multiple markets and endpoints, you will need to upgrade your Parcl Labs API account from free to starter to get additional credits. You can easily upgrade at any time by visiting your [Parcl Labs dashboard](https://dashboard.parcllabs.com/login), clicking "Upgrade Now" ($99, no commitment). This will unlock more credits immediately.

Run in Collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/introduction/download_data.ipynb)

**Table of Contents Of Endpoint Data Downloads:**
1. **Market Metrics:**
   - [Housing Event Counts](https://docs.parcllabs.com/reference/housing_event_counts_v1_market_metrics__parcl_id__housing_event_counts_get-1)
   - [Housing Event Prices](https://docs.parcllabs.com/reference/housing_event_prices_v1_market_metrics__parcl_id__housing_event_prices_get-1)
   - [Housing Stock](https://docs.parcllabs.com/reference/housing_stock_v1_market_metrics__parcl_id__housing_stock_get-1)
   - [All Cash Transactions](https://docs.parcllabs.com/reference/all_cash_v1_market_metrics__parcl_id__all_cash_get)
   - [Housing Event Property Attributes](https://docs.parcllabs.com/reference/housing_event_property_attributes_v1_market_metrics__parcl_id__housing_event_property_attributes_get)
2. **For Sale Market Metrics:**
   - [New Listings Rolling Counts](https://docs.parcllabs.com/reference/new_listings_rolling_counts_v1_for_sale_market_metrics__parcl_id__new_listings_rolling_counts_get-1)
   - [For Sale Inventory](https://docs.parcllabs.com/reference/for_sale_inventory_v1_for_sale_market_metrics__parcl_id__for_sale_inventory_get)
   - [For Sale Inventory Price Changes](https://docs.parcllabs.com/reference/for_sale_inventory_price_changes_v1_for_sale_market_metrics__parcl_id__for_sale_inventory_price_changes_get)
3. **Rental Market Metrics:**
   - [Gross Yield](https://docs.parcllabs.com/reference/gross_yield_v1_rental_market_metrics__parcl_id__gross_yield_get-1)
   - [Rental Units Concentration](https://docs.parcllabs.com/reference/rental_units_concentration_v1_rental_market_metrics__parcl_id__rental_units_concentration_get)
   - [New Listings for Rent Rolling Counts](https://docs.parcllabs.com/reference/new_listings_for_rent_rolling_counts_v1_rental_market_metrics__parcl_id__new_listings_for_rent_rolling_counts_get-1)
4. **Investor Metrics:**
   - [Housing Event Counts](https://docs.parcllabs.com/reference/housing_event_counts_v1_investor_metrics__parcl_id__housing_event_counts_get-1)
   - [Purchase to Sale Ratio](https://docs.parcllabs.com/reference/purchase_to_sale_ratio_v1_investor_metrics__parcl_id__purchase_to_sale_ratio_get-1)
   - [New Listings for Sale Rolling Counts](https://docs.parcllabs.com/reference/new_listings_for_sale_rolling_counts_v1_investor_metrics__parcl_id__new_listings_for_sale_rolling_counts_get-1)
   - [Housing Stock Ownership](https://docs.parcllabs.com/reference/housing_stock_ownership_v1_investor_metrics__parcl_id__housing_stock_ownership_get-1)
   - [Housing Event Prices](https://docs.parcllabs.com/reference/housing_event_prices_v1_investor_metrics__parcl_id__housing_event_prices_get)
5. **Portfolio Metrics:**
   - [Single Family Housing Event Counts](https://docs.parcllabs.com/reference/sf_housing_event_counts_v1_portfolio_metrics__parcl_id__sf_housing_event_counts_get)
   - [Single Family Housing Stock Ownership](https://docs.parcllabs.com/reference/sf_housing_stock_ownership_v1_portfolio_metrics__parcl_id__sf_housing_stock_ownership_get-1)
   - [New Listings for Sale Rolling Counts](https://docs.parcllabs.com/reference/sf_new_listings_for_sale_rolling_counts_v1_portfolio_metrics__parcl_id__sf_new_listings_for_sale_rolling_counts_get)
   - [New Listings for Rent Rolling Counts](https://docs.parcllabs.com/reference/sf_new_listings_for_rent_rolling_counts_v1_portfolio_metrics__parcl_id__sf_new_listings_for_rent_rolling_counts_get)

In [ ]:
import os
import sys
import json
import subprocess
from datetime import datetime
from urllib.request import urlopen

# Check if running in Google Colab
if "google.colab" in sys.modules:
    from google.colab import userdata
    # Install required packages
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'parcllabs', 'plotly', 'kaleido'])
    # Retrieve API key from user data
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    # Retrieve API key from environment variable
    api_key = os.getenv('PARCL_LABS_API_KEY')

# Ensure the API key is available
if not api_key:
    raise ValueError("API key for Parcl Labs is not set. Please set the 'PARCL_LABS_API_KEY' environment variable or provide it in Google Colab's userdata.")

In [ ]:
# Import parcllabs and create a client
import parcllabs
from parcllabs import ParclLabsClient

# Print the version of parcllabs
print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# Initialize the Parcl Labs client with a limit for all calls
client = ParclLabsClient(api_key, limit=12)

In [ ]:
# Let's get some sample markets to download data for; everything starts with a search.
# We'll retrieve the top 12 most populous metropolitan statistical areas (MSAs) in the country.
# There are many different ways to search for markets. The markets defined here will be used to set the fields for downloaded data.
# For more details, please refer to the Search Notebook: https://github.com/ParclLabs/parcllabs-examples/blob/main/python/introduction/search.ipynb

markets = client.search_markets.retrieve(
    location_type='CBSA',  # Used: CBSA stands for Core-Based Statistical Area, which includes metropolitan areas. Example: 'CITY'
    sort_by='TOTAL_POPULATION',  # Used: We're sorting the results by total population to get the largest metros. Example: 'MEDIAN_INCOME'
    sort_order='DESC',  # Used: DESC means descending order, so the largest populations come first. Example: 'ASC'
    as_dataframe=True  # Used: We want the results returned as a DataFrame for easy handling in Python.

    # Other available parameters:
    # region='EAST_NORTH_CENTRAL',  # Specify the region if needed.
    # state_abbreviation='CA',  # Filter by state abbreviation.
    # query='San Francisco',  # Search by market name or other keywords.
)

# Let's store the unique identifiers (parcl_ids) of these markets for later use.
# We'll need these IDs to download specific data for each market in subsequent steps.
parcl_ids = markets['parcl_id'].tolist()

# Display the top 5 markets to verify our search results.
markets.head(5)

In [ ]:
# Set the start date for the entire notebook.
# This date defines the beginning of the data range we want to retrieve.
# You can change this date to alter the range of data that is returned.
START_DATE = '2023-01-01'

# Placeholder for the end date parameter.
# END_DATE = 'YYYY-MM-DD'  # Uncomment and set the end date if needed.

# For this example, we're only setting the start date to get data from the past year.

In [ ]:
# Download data from the Housing Event Counts endpoint
results_housing_event_counts = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_housing_event_counts = results_housing_event_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_event_counts.head()

In [ ]:
# Save the data to the housing_event_counts CSV file locally
results_housing_event_counts.to_csv('housing_event_counts.csv', index=False)

In [ ]:
# Download data from the Housing Stock endpoint
results_housing_stock = client.market_metrics_housing_stock.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_housing_stock = results_housing_stock.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_stock.head()

In [ ]:
# Save the data to the housing_stock CSV file locally
results_housing_stock.to_csv('housing_stock.csv', index=False)

In [ ]:
# Download data from the Housing Event Prices endpoint
results_housing_event_prices = client.market_metrics_housing_event_prices.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_housing_event_prices = results_housing_event_prices.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_event_prices.head()

In [ ]:
# Save the data to the housing_event_prices CSV file locally
results_housing_event_prices.to_csv('housing_event_prices.csv', index=False)

In [ ]:
# Download data from the Housing Event Property Attributes endpoint
results_housing_event_property_attributes = client.market_metrics_housing_event_property_attributes.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_housing_event_property_attributes = results_housing_event_property_attributes.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_event_property_attributes.head()

In [ ]:
# Save the data to the housing_event_property_attributes CSV file locally
results_housing_event_property_attributes.to_csv('housing_event_property_attributes.csv', index=False)

In [ ]:
# Download data from the All Cash endpoint
results_all_cash = client.market_metrics_all_cash.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_all_cash = results_all_cash.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_all_cash.head()

In [ ]:
# Save the data to the all_cash CSV file locally
results_all_cash.to_csv('all_cash.csv', index=False)

In [ ]:
# Download data from the Rental Market Metrics Gross Yield endpoint
results_gross_yield = client.rental_market_metrics_gross_yield.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_gross_yield = results_gross_yield.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_gross_yield.head()

In [ ]:
# Save the data to the gross_yield CSV file locally
results_gross_yield.to_csv('gross_yield.csv', index=False)

In [ ]:
# Download data from the Rental Market Metrics Rental Units Concentration endpoint
results_rental_units_concentration = client.rental_market_metrics_rental_units_concentration.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_rental_units_concentration = results_rental_units_concentration.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_rental_units_concentration.head()

In [ ]:
# Save the data to the rental_units_concentration CSV file locally
results_rental_units_concentration.to_csv('rental_units_concentration.csv', index=False)

In [ ]:
# Download data from the Rental Market Metrics New Listings for Rent Rolling Counts endpoint
results_new_listings_for_rent_rolling_counts = client.rental_market_metrics_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_rent_rolling_counts = results_new_listings_for_rent_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_rent_rolling_counts.head()

In [ ]:
# Save the data to the new_listings_for_rent_rolling_counts CSV file locally
results_new_listings_for_rent_rolling_counts.to_csv('new_listings_for_rent_rolling_counts.csv', index=False)

In [ ]:
# Download data from the For Sale Market Metrics New Listings Rolling Counts endpoint
results_new_listings_rolling_counts = client.for_sale_market_metrics_new_listings_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_rolling_counts = results_new_listings_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_rolling_counts.head()

In [ ]:
# Save the data to the new_listings_rolling_counts CSV file locally
results_new_listings_rolling_counts.to_csv('new_listings_rolling_counts.csv', index=False)

In [ ]:
# Download data from the For Sale Market Metrics For Sale Inventory Price Changes endpoint
results_for_sale_inventory_price_changes = client.for_sale_market_metrics_for_sale_inventory_price_changes.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # property_type='ALL_PROPERTIES',  # Uncomment and set the property type if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_for_sale_inventory_price_changes = results_for_sale_inventory_price_changes.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_for_sale_inventory_price_changes.head()

In [ ]:
# Save the data to the for_sale_inventory_price_changes CSV file locally
results_for_sale_inventory_price_changes.to_csv('for_sale_inventory_price_changes.csv', index=False)

In [ ]:
# Download data from the Investor Metrics Housing Event Counts endpoint
results_investor_housing_event_counts = client.investor_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_investor_housing_event_counts = results_investor_housing_event_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_investor_housing_event_counts.head()

In [ ]:
# Save the data to the investor_housing_event_counts CSV file locally
results_investor_housing_event_counts.to_csv('investor_housing_event_counts.csv', index=False)

In [ ]:
# Download data from the Investor Metrics Purchase to Sale Ratio endpoint
results_purchase_to_sale_ratio = client.investor_metrics_purchase_to_sale_ratio.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_purchase_to_sale_ratio = results_purchase_to_sale_ratio.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_purchase_to_sale_ratio.head()

In [ ]:
# Save the data to the purchase_to_sale_ratio CSV file locally
results_purchase_to_sale_ratio.to_csv('purchase_to_sale_ratio.csv', index=False)

In [ ]:
# Download data from the Investor Metrics New Listings for Sale Rolling Counts endpoint
results_new_listings_for_sale_rolling_counts = client.investor_metrics_new_listings_for_sale_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_sale_rolling_counts = results_new_listings_for_sale_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_sale_rolling_counts.head()

In [ ]:
# Save the data to the new_listings_for_sale_rolling_counts CSV file locally
results_new_listings_for_sale_rolling_counts.to_csv('new_listings_for_sale_rolling_counts.csv', index=False)

In [ ]:
# Download data from the Investor Metrics Housing Stock Ownership endpoint
results_housing_stock_ownership = client.investor_metrics_housing_stock_ownership.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_housing_stock_ownership = results_housing_stock_ownership.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_housing_stock_ownership.head()

In [ ]:
# Save the data to the housing_stock_ownership CSV file locally
results_housing_stock_ownership.to_csv('housing_stock_ownership.csv', index=False)

In [ ]:
# Download data from the Investor Metrics Housing Event Prices endpoint
results_investor_housing_event_prices = client.investor_metrics_housing_event_prices.retrieve_many(
    parcl_ids=parcl_ids,
    start_date=START_DATE,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_investor_housing_event_prices = results_investor_housing_event_prices.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_investor_housing_event_prices.head()

In [ ]:
# Save the data to the investor_housing_event_prices CSV file locally
results_investor_housing_event_prices.to_csv('investor_housing_event_prices.csv', index=False)

In [ ]:
# Download data from the Portfolio Metrics Single Family Housing Event Counts endpoint
results_sf_housing_event_counts = client.portfolio_metrics_sf_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_sf_housing_event_counts = results_sf_housing_event_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_sf_housing_event_counts.head()

In [ ]:
# Save the data to the sf_housing_event_counts CSV file locally
results_sf_housing_event_counts.to_csv('sf_housing_event_counts.csv', index=False)

In [ ]:
# Download data from the Portfolio Metrics Single Family Housing Stock Ownership endpoint
results_sf_housing_stock_ownership = client.portfolio_metrics_sf_housing_stock_ownership.retrieve_many(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_sf_housing_stock_ownership = results_sf_housing_stock_ownership.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_sf_housing_stock_ownership.head()

In [ ]:
# Save the data to the sf_housing_stock_ownership CSV file locally
results_sf_housing_stock_ownership.to_csv('sf_housing_stock_ownership.csv', index=False)

In [ ]:
# Download data from the Portfolio Metrics New Listings for Sale Rolling Counts endpoint
results_new_listings_for_sale_rolling_counts = client.portfolio_metrics_new_listings_for_sale_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_sale_rolling_counts = results_new_listings_for_sale_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_sale_rolling_counts.head()

In [ ]:
# Save the data to the portfolio_metrics_new_listings_for_sale_rolling_counts CSV file locally
results_new_listings_for_sale_rolling_counts.to_csv('portfolio_metrics_new_listings_for_sale_rolling_counts.csv', index=False)

In [ ]:
# Download data from the Portfolio Metrics New Listings for Rent Rolling Counts endpoint
results_new_listings_for_rent_rolling_counts = client.portfolio_metrics_sf_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    # end_date=END_DATE,  # Uncomment and set the end date if needed.
    # portfolio_size='ALL_PORTFOLIOS',  # Uncomment and set the portfolio size if needed.
    # Options: 'PORTFOLIO_2_TO_9', 'PORTFOLIO_10_TO_99', 'PORTFOLIO_100_TO_999', 'PORTFOLIO_1000_PLUS', 'ALL_PORTFOLIOS'
    as_dataframe=True
)

# Include the parcl_id and market name in the DataFrame
results_new_listings_for_rent_rolling_counts = results_new_listings_for_rent_rolling_counts.merge(markets[['parcl_id', 'name']], on='parcl_id')

# Display the first few rows to verify the results
results_new_listings_for_rent_rolling_counts.head()

In [ ]:
# Save the data to the portfolio_metrics_new_listings_for_rent_rolling_counts CSV file locally
results_new_listings_for_rent_rolling_counts.to_csv('portfolio_metrics_new_listings_for_rent_rolling_counts.csv', index=False)

In [ ]:
# For downloading Price Feed data, please refer to the Price Feed Notebook:
# https://github.com/ParclLabs/parcllabs-examples/blob/main/python/introduction/price_feed.ipynb